In [1]:
# Importing all necessary libraries
import os
import pandas as pd  # For handling data in tables (DataFrames)
import SimpleITK as sitk
from radiomics import featureextractor  
# The main PyRadiomics class
import logging

# --- Configuration ---
# Setting up a logger to show PyRadiomics progress and warnings, but not too much detail

logging.getLogger('radiomics').setLevel(logging.WARNING)

# Setting the base path to our data
# This is where the images and masks are stored
data_base_path = '../Data/NSCLC-Radiomics'

print("Libraries imported and configuration set.")

Libraries imported and configuration set.


In [2]:
# --- Extractor Setup ---

# First, initialize an empty extractor
from radiomics import featureextractor
import SimpleITK as sitk

extractor = featureextractor.RadiomicsFeatureExtractor()

# Now, we manually set the parameters we want to customize.
# This is the modern and correct way to do it.
print("Customizing extractor settings...")
extractor.settings['binWidth'] = 25
extractor.settings['resampledPixelSpacing'] = [1, 1, 1]  # Resample to 1x1x1 mm voxels
extractor.settings['interpolator'] = 'sitkBSpline'      # Use B-Spline interpolation
# For lung RTSTRUCT files, the primary tumor (GTV) is almost always label '1'
extractor.settings['label'] = 1

# For now, let's enable all feature classes
extractor.enableAllFeatures()

print("PyRadiomics feature extractor initialized and configured successfully.")

Customizing extractor settings...
PyRadiomics feature extractor initialized and configured successfully.


In [3]:
# --- Main Loop ---

# This will store the results for all patients
all_features = []

# Geting a sorted list of all patient folders
patient_folders = sorted([f for f in os.listdir(data_base_path) if os.path.isdir(os.path.join(data_base_path, f))])

print(f"Found {len(patient_folders)} patients. Starting extraction...")

# Looping through each patient folder

for patient_id in patient_folders:
    print(f"Processing patient: {patient_id}...")
    patient_path = os.path.join(data_base_path, patient_id)
    
    # --- Step 1: Find Image and Mask files (using our robust logic from Task 03) ---

    image_series_path = ''
    mask_file_path = ''

    # Walking through all the directories and files inside the patient folder
    # to find the image series and mask file
    for root, dirs, files in os.walk(patient_path):
        if 'SEGMENTATION' in root.upper():
            # If I find a segmentation folder, I assume it contains the mask
            if files: mask_file_path = os.path.join(root, files[0])

        if len(files) > 50 and files[0].endswith('.dcm'):
            # If I find a folder with more than 50 DICOM files, I assume it's the image series
            image_series_path = root

    # --- Step 2: Load and Resample (if paths were found) ---

    if image_series_path and mask_file_path:
        try:
            
            # Load image
            image_reader = sitk.ImageSeriesReader()
            dicom_names = image_reader.GetGDCMSeriesFileNames(image_series_path)
            image_reader.SetFileNames(dicom_names)
            image_3d = image_reader.Execute()

            # Load mask
            mask_3d = sitk.ReadImage(mask_file_path)

            # Resample mask to match image
            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(image_3d)
            resampler.SetInterpolator(sitk.sitkNearestNeighbor)
            resampled_mask_3d = resampler.Execute(mask_3d)

            # --- Step 3: Extract Features ---
            # The extractor takes the image and the resampled mask
            # It's crucial to use the original image and the resampled mask, not the original mask
            feature_vector = extractor.execute(image_3d, resampled_mask_3d, label=255)
            
            # Add the patient ID to our results
            feature_vector['PatientID'] = patient_id
            all_features.append(feature_vector)
            
            print(f"  > Success! Extracted {len(feature_vector)-1} features.")

        except Exception as e:
            print(f"  > FAILED for patient {patient_id}. Error: {e}")
    else:
        print(f"  > FAILED for patient {patient_id}. Could not find image or mask path.")

print("\nFeature extraction complete.")

Found 422 patients. Starting extraction...
Processing patient: LUNG1-001...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 144 features.
Processing patient: LUNG1-002...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 144 features.
Processing patient: LUNG1-003...


...

  > Success! Extracted 144 features.
Processing patient: LUNG1-421...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 144 features.
Processing patient: LUNG1-422...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 144 features.

Feature extraction complete.


In [5]:
# --- Saving Results ---

# Convertint the list of feature dictionaries into a pandas DataFrame
# A DataFrame is like a spreadsheet or a table

features_df = pd.DataFrame(all_features)

# Setting the order of the columns to get the PatientID as the first column for easy reference

cols = ['PatientID'] + [col for col in features_df.columns if col != 'PatientID']

features_df = features_df[cols]

# Setting the output path for the output file
output_path = '../Results/radiomics_features.csv'

# Saving the DataFrame to a CSV file
features_df.to_csv(output_path, index=False)

print(f"Successfully saved {len(features_df)} patient records to:")
print(os.path.abspath(output_path))

# Displaying the first 5 rows of the new table
features_df.head()

Successfully saved 421 patient records to:
d:\Thesis_Project\Results\radiomics_features.csv


,PatientID,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LUNG1-001,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},b4175e0d6ddff17507e30c05c6e7302c2b9668f4,3D,...,974.3968868560783,0.0012217040883470618,8.354220960978468,0.09712593876372039,340295.02098699583,11.298544945318655,1.4698658219887565e-05,6179.411678264663,0.04832815861023239,0.18810135440340525
1,LUNG1-002,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5c66c70353901fd238191b88a2584cd8c3f645e5,3D,...,801.965399244845,0.0014035581231717656,8.10932092162548,0.1394304445325397,38878.35473106718,4.0953664654888655,5.86909917257305e-05,4744.542628527841,0.09340931019816005,0.25842490143353464
2,LUNG1-003,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9a2b123cabcc00593e4255d41cbef78f0cb63630,3D,...,1338.700821914673,0.003880429318442169,8.361764040606204,0.07780431526535239,460685.553449732,4.358217959833733,4.8227735366151494e-05,6626.985903168955,0.07202148319412406,1.569060887336299
3,LUNG1-004,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e20a402cf62f481dc190b62e20f6d514805b904c,3D,...,1324.4810295868097,0.0016392216745715886,8.302489177323388,0.07513746666227651,480820.22005284694,6.094141973470173,2.4220275723587345e-05,4522.199680553571,0.03530010873299323,0.2899886551495625
4,LUNG1-005,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},08c0636c812e2424fe72a17d221597001b7201b5,3D,...,2355.961306070771,0.0010593290239718153,8.572751786965446,0.1565344284070606,15589.066270885198,0.8832821192206137,8.695905954928009e-05,30662.484047544334,0.03122887740949193,5.1733135818012785
